In [17]:
import os
import torch
import GPUtil
import numpy as np
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
# from models.POD import PODNonlinearModel
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [18]:
from collections import Counter

In [19]:
import matplotlib.pyplot as plt
from vecopsciml.operators.zero_order import Mx, My

In [20]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear_1000.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/fourier_model')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/fourier_model


In [21]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear/non_linear_1000.pkl


In [22]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [23]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [24]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [25]:
data = y_train.values.detach().to('cpu')
n_modes = 100
relative_energy = 0.0

fft_data = torch.fft.fft2(data)
# fft_data_shifted = torch.fft.fftshift(fft_data)
fft_data_shifted = (fft_data)
energy = torch.abs(fft_data_shifted)**2
energy_flattened = energy.flatten(1, 3)

for mode_i in range(n_modes):

    top_energetic = energy_flattened[:, :mode_i]
    relative_energy = (torch.sum(torch.sum(top_energetic))/torch.sum(torch.sum(energy_flattened))).item()

    print(mode_i, '-->', relative_energy)

0 --> 0.0
1 --> 0.9805648326873779
2 --> 0.983593225479126
3 --> 0.9844560027122498
4 --> 0.9849163889884949
5 --> 0.9852505922317505
6 --> 0.9855523109436035
7 --> 0.9858865141868591
8 --> 0.9863457679748535
9 --> 0.9872096180915833
10 --> 0.9902372360229492
11 --> 0.9932528734207153
12 --> 0.9932627081871033
13 --> 0.9932650327682495
14 --> 0.993266761302948
15 --> 0.9932681322097778
16 --> 0.9932698011398315
17 --> 0.9932708144187927
18 --> 0.9932726621627808
19 --> 0.9932757019996643
20 --> 0.9932872653007507
21 --> 0.9941490888595581
22 --> 0.9941521286964417
23 --> 0.9941530227661133
24 --> 0.9941537380218506
25 --> 0.9941538572311401
26 --> 0.9941543340682983
27 --> 0.994154691696167
28 --> 0.9941554069519043
29 --> 0.9941568374633789
30 --> 0.9941604137420654
31 --> 0.9946208596229553
32 --> 0.9946224689483643
33 --> 0.994623064994812
34 --> 0.9946233630180359
35 --> 0.9946235418319702
36 --> 0.9946237802505493
37 --> 0.9946239590644836
38 --> 0.9946243166923523
39 --> 0.994624

In [26]:
##### No sirve para lo que queremos. Al buscar asi los modos mas energeticos, la NN sobreentrena. Lo haremos escogiendo los N primeros en vez de los N mas energéticos. 

def modes_base(data, n_modes):

    # FFT decomposition and obtain energy of each mode
    fft_data = torch.fft.fft2(data)
    fft_data_shifted = torch.fft.fftshift(fft_data)
    energy = torch.abs(fft_data_shifted)
    energy_flattened = energy.flatten(1, 3)

    # Get the n_modes more energetic modes and their indices
    top_energetic = torch.topk(energy_flattened, n_modes).indices
    top_energetic_indices, _ = zip(*Counter(top_energetic.flatten()).most_common(n_modes))
    top_energetic_indices = list(map(int, top_energetic_indices))

    # Create an empty template to include the modes
    filtered_modes = torch.zeros_like(energy, dtype=torch.complex64)
    filtered_modes.flatten(1, 3)[:, top_energetic_indices] = fft_data_shifted.flatten(1, 3)[:, top_energetic_indices]

    # Return the base with the 'n_modes' most energetic modes
    return top_energetic_indices, filtered_modes

def reconstruct_data(coefficients_shifted):
    
    # Compute inverse FFT and reconstruct data
    filtered_modes_base = torch.fft.ifftshift(coefficients_shifted)
    reconstructed_data = torch.real(torch.fft.ifft2(filtered_modes_base))

    return reconstructed_data

In [27]:
def modes_base(data, n_modes):

    # FFT decomposition and obtain energy of each mode
    fft_data = torch.fft.fft2(data)
    energy = torch.abs(fft_data)
    energy_flattened = energy.flatten(1, 3)

    # Get the n_modes more energetic modes and their indices
    top_energetic = energy_flattened[:, :n_modes]

    # Create an empty template to include the modes
    filtered_modes = torch.zeros_like(energy, dtype=torch.complex64)
    filtered_modes.flatten(1, 3)[:, :n_modes] = fft_data.flatten(1, 3)[:, :n_modes]

    # Return the base with the 'n_modes' most energetic modes
    return filtered_modes

def reconstruct_data(coefficients_filtered):
    
    # Compute inverse FFT and reconstruct data
    reconstructed_data = torch.real(torch.fft.ifft2(coefficients_filtered))

    return reconstructed_data

In [28]:
num_modes = 90

_ = modes_base(data=y_train.values, n_modes=num_modes)
reconstructed_data = reconstruct_data(_)

In [29]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, num_modes]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [30]:
import torch
import torch.nn as nn

from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My

class Encoder(nn.Module):

    def __init__(self, input_size, hidden_layer_1_size, hidden_layer_2_size, latent_space_size):
        super(Encoder, self).__init__()

        # Parameters
        self.in_size = torch.tensor(input_size)
        self.h1_size = hidden_layer_1_size
        self.h2_size = hidden_layer_2_size
        self.ls_size = latent_space_size

        # Architecture
        self.flatten_layer = nn.Flatten(start_dim=1, end_dim=-1)
        self.hidden1_layer = nn.Linear(torch.prod(self.in_size), self.h1_size)
        self.hidden2_layer = nn.Linear(self.h1_size, self.h2_size)
        self.latent_space_layer = nn.Linear(self.h2_size, self.ls_size)
    
    def forward(self, X):
        
        X = self.flatten_layer(X)
        X = torch.sigmoid(self.hidden1_layer(X))
        X = torch.sigmoid(self.hidden2_layer(X))
        latent_space_output = (self.latent_space_layer(X))

        return latent_space_output
    
class Explanatory(nn.Module):

    def __init__(self, input_size, n_filters, hidden_layer_size, output_size):
        super(Explanatory, self).__init__()

        # Parameters
        self.in_size = torch.tensor(input_size)
        self.n_filters = n_filters
        self.h_layer = hidden_layer_size
        self.out_size = torch.tensor(output_size)

        # Architecture
        self.conv_expand_layer = nn.Conv2d(in_channels=1, out_channels=self.n_filters, kernel_size=1)
        self.flatten_layer = nn.Flatten(start_dim=1, end_dim=-1)
        self.hidden_layer = nn.Linear(n_filters*torch.prod(self.in_size), n_filters*torch.prod(self.out_size))
        self.conv_converge_layer = nn.Conv2d(in_channels=n_filters, out_channels=1, kernel_size=1)
        
    def forward(self, X):
        
        X = torch.sigmoid(self.conv_expand_layer(X))
        X = self.flatten_layer(X)
        X = self.hidden_layer(X)
        X = X.view(X.size(0), self.n_filters, self.out_size[1], self.out_size[2])
        explanatory_output = self.conv_converge_layer(X)

        return explanatory_output
    
class FFTNonlinearModel(nn.Module):
    
    def __init__(self, input_size, predictive_layers, FFT_modes_base, output_predictive_size, explanatory_input_size, explanatory_layers, output_explanatory_size, n_filters):
        
        super(FFTNonlinearModel, self).__init__()

        # Parameters
        self.in_size = input_size
        self.pred_size = predictive_layers
        self.out_pred_size = output_predictive_size
        
        self.in_exp_size = explanatory_input_size
        self.exp_size = explanatory_layers
        self.out_exp_size = output_explanatory_size

        self.n_filters = n_filters

        # Architecture
        self.encoder = Encoder(self.in_size, self.pred_size[0], self.pred_size[1], 2*self.pred_size[2])
        self.base_indices = FFT_modes_base
        self.explanatory = Explanatory(self.in_exp_size, self.n_filters, self.exp_size[0], self.out_exp_size)
        
    def forward(self, X):

        # Predictive network
        X = self.encoder(X)

        # Manipulating output to obtain real and complex part
        output_predictive = X.view(X.size(0), self.pred_size[2], 2)
        real = output_predictive[..., 0]
        imag = output_predictive[..., 1]
    
        # Reconstruction with FFT and manipulation of prediction output
        base = torch.zeros((X.size(0), *self.out_pred_size), dtype=torch.complex64).to(DEVICE)
        base.flatten(1, 3)[:, :self.base_indices] = torch.complex(real, imag)

        u = reconstruct_data(base).to(DEVICE)        
        um = Mx(My(TensOps(u, space_dimension=2, contravariance=0, covariance=0))).values

        # Explanatory network
        K = self.explanatory(um)
        
        return u, K

In [31]:
# Load model and the optimizer
model = FFTNonlinearModel(input_shape, predictive_layers, num_modes, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 10000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 7.557e+08, Test loss: 9.966e+08, MSE(e): 7.517e+01, MSE(pi1): 3.743e+02, MSE(pi2): 3.031e+01, MSE(pi3): 2.403e+00
Epoch 100, Train loss: 6.083e+07, Test loss: 8.761e+07, MSE(e): 6.081e+00, MSE(pi1): 4.760e-01, MSE(pi2): 2.585e+00, MSE(pi3): 1.246e-01
Epoch 200, Train loss: 2.445e+07, Test loss: 4.347e+07, MSE(e): 2.443e+00, MSE(pi1): 4.649e-01, MSE(pi2): 1.128e+00, MSE(pi3): 1.249e-01
Epoch 300, Train loss: 6.937e+06, Test loss: 9.968e+06, MSE(e): 6.921e-01, MSE(pi1): 5.306e-01, MSE(pi2): 3.998e-01, MSE(pi3): 1.019e-01
Epoch 400, Train loss: 2.969e+06, Test loss: 3.525e+06, MSE(e): 2.960e-01, MSE(pi1): 2.161e-01, MSE(pi2): 1.942e-01, MSE(pi3): 6.402e-02
Epoch 500, Train loss: 2.666e+05, Test loss: 5.043e+05, MSE(e): 2.636e-02, MSE(pi1): 1.008e-01, MSE(pi2): 1.531e-02, MSE(pi3): 1.943e-02
Epoch 600, Train loss: 1.075e+05, Test loss: 2.202e+05, MSE(e): 1.053e-02, MSE(pi1): 7.775e-02, MSE(pi2): 6.603e-03, MSE(pi3): 1.403e-02
Epoch 700, 

In [32]:
# Parametros de entrenamiento
start_epoch = 9000
n_epochs = 100000

batch_size = 64
n_checkpoints = 100

second_lr = 1e-2

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 9000.
Epoch 9000, Train loss: 2.774e+03, Test loss: 7.989e+03, MSE(e): 2.160e-04, MSE(pi1): 4.567e-02, MSE(pi2): 1.439e-04, MSE(pi3): 1.579e-03
Epoch 9100, Train loss: 2.091e+03, Test loss: 8.167e+03, MSE(e): 1.732e-04, MSE(pi1): 2.634e-02, MSE(pi2): 1.320e-04, MSE(pi3): 9.560e-04
Epoch 9200, Train loss: 5.782e+03, Test loss: 1.343e+04, MSE(e): 4.820e-04, MSE(pi1): 7.729e-02, MSE(pi2): 2.768e-04, MSE(pi3): 1.889e-03
Epoch 9300, Train loss: 2.698e+03, Test loss: 7.062e+03, MSE(e): 2.216e-04, MSE(pi1): 3.519e-02, MSE(pi2): 1.578e-04, MSE(pi3): 1.294e-03
Epoch 9400, Train loss: 2.078e+03, Test loss: 7.076e+03, MSE(e): 1.724e-04, MSE(pi1): 2.663e-02, MSE(pi2): 1.238e-04, MSE(pi3): 8.793e-04
Epoch 9500, Train loss: 3.635e+03, Test loss: 1.007e+04, MSE(e): 3.210e-04, MSE(pi1): 3.160e-02, MSE(pi2): 1.951e-04, MSE(pi3): 1.092e-03
Epoch 9600, Train loss: 6.040e+03, Test loss: 1.174e+04, MSE(e): 5.677e-04, MSE(pi1): 2.332e-02, MSE(pi2): 2.936e-04, MSE(p